In [2]:
import tensorflow.compat.v1 as tf 
import matplotlib.pyplot as plt
from tqdm import tqdm
import lucid_kietzmannlab.modelzoo.vision_models as models
from lucid_kietzmannlab.misc.io import show
import lucid_kietzmannlab.optvis.objectives as objectives
import lucid_kietzmannlab.optvis.param as param
import lucid_kietzmannlab.optvis.render as render
import lucid_kietzmannlab.optvis.transform as transform
from IPython.display import clear_output
from ipywidgets import interact, Dropdown, IntSlider


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.1 MB/s eta 0:00:00


## A notebook to maximally activate certain neurons in the Alex Net neural network 

### Choose the alexnet model

In [3]:
model = models.AlexNet()
model.load_graphdef()

### In the code block below a user can interactively choose the layer to maximally activate the neuron of and then visualize it for a certain channel

In [4]:
layer_name_list = [node.name for node in model.graph_def.node if 'input' not in node.name]

# Get the shape of each layer
with tf.Graph().as_default() as graph:
    # Import the model
    tf.import_graph_def(model.graph_def, name='')

    # Get the shape of each tensor
    layer_shape_dict = {}
    with tf.compat.v1.Session() as sess:
        for layer_name in layer_name_list:
            try:
                tensor_shape = sess.graph.get_tensor_by_name(f'{layer_name}:0').shape
                
                if tensor_shape != ():
                  
                    if tensor_shape[0] is None:
                       
                       layer_shape_dict[layer_name] = tensor_shape
            except KeyError:
                # Handle the case where the tensor is not found
                layer_shape_dict[layer_name] = None

# Print the shape of each layer
#for layer_name, tensor_shape in layer_shape_dict.items():
#    print(f'Layer: {layer_name}, Shape: {tensor_shape}')
C = lambda neuron: objectives.channel(*neuron)    
def visualize(layer_name, channel):
    # Check if the layer exists in the shape dictionary
    if layer_name in layer_shape_dict:
        # Check if the selected channel is within bounds
        print(layer_shape_dict[layer_name])
        max_channel = layer_shape_dict[layer_name][-1] - 1
        if 0 <= channel <= max_channel:
            clear_output(wait=True)
            # Render visualization for the selected layer and channel
            try:
               _ = render.render_vis(model, C((layer_name, channel)))
            except Exception:
                print('No gradients for this layer')   

def visualize_all():
    # Check if the layer exists in the shape dictionary
    layer_name = layer_dropdown.value
    if layer_name in layer_shape_dict:
            # Check if the selected channel is within bounds
            try:
               image_channel = {} 
               for channel in tqdm(range(channel_slider.max)):    
                  images = render.render_vis(model, C((layer_name, channel)), verbose = False)
                  image_channel[channel] = images 
            except Exception:
                print('No gradients for this layer')    
                              
def plot_images(image_channel):
    num_channels = len(image_channel)
    num_rows = 4
    num_cols = num_channels // num_rows + (num_channels % num_rows > 0)
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 12))
    for i, (channel, images) in enumerate(image_channel.items()):
        row = i // num_cols
        col = i % num_cols
        axs[row, col].imshow(images[0])  # Assuming images[0] is the visualization result
        axs[row, col].set_title(f"Channel {channel}")
        axs[row, col].axis('off')
    plt.tight_layout()
    plt.show()
# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
        
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     

       

interactive(children=(Dropdown(description='Layer:', options=('Placeholder', 'Conv2D', 'BiasAdd', 'Reshape', '…

<function __main__.visualize(layer_name, channel)>

In the code below we make a non-interactive plot of all the channels at that layer at once (Memory intensive)

In [5]:
image_channel = visualize_all()
if image_channel:
    plot_images(image_channel)    

  5%|▍         | 6/127 [00:17<05:56,  2.94s/it]